Accuracy and speed:
- in math, stuff can be 100% accurate, but for calculating with computers we have accuracy (how close something is to the analytical answer or a more accurate calculation) limitations
- we also have to think about speed
    - when things take only a few seconds it doesn't really matter (in this setting) but for a long simulation something like 3 vs 6 months is a huge deal
- there are often tradeoffs between accuracy and speed
- figuring out how accurate something is is non trivial because we don't know the "correct" answer
- machine precision - floats are stored using a fixed number of bits
    - 32 bit/single precision:
        - 1 bit sign, 8 bits exponent, 24 bits (23 stored) significand
        - range of 2^7 -1 in exponent so about 10^38
        - precision of 6 significant digits
        - good enough for most real world applications (finance etc)
    - 64 bit/double precision:
        - 1 bit sign, 11 bits exponent, 53 bits (52 stored) significand
        - range of 2^10 -1 in exponent so about 10^308
        - precision of 15 significant digits
        - better for astrophysical applications
        - standard python floats now
    - most chips are 64 bit now so it can use doubles much more efficiently
    - can use even higher precision but efficiency will be reduced 

In [7]:
x=1.0
eps=1.0
while not x+eps==x: #this will be true once eps is smaller than the machine precision
    eps=0.5*eps
print(2*eps)

2.220446049250313e-16


- machine precision cont:
    - can see that once you go past 10^-15 it's effectively zero
- overflow error:
    - if python gets to a number greater than 10^308 it will set it to infinity

In [20]:
1e308

1e+308

In [22]:
10*1e308

inf

- underflow error:
    - past the smallest possible float, python will just set it to 0.0
- rounding/accuracy error:
    - don't ever check if floats are exactly equal, instead check within an error
    - ex: (abs(x-2.50) < eps)
    - behaves much likes a measurement error in a physics lab where it can compound with more values
    - sigma^2=sqrt(sigma1^2+sigma2^2)

In [29]:
b=0.1 #this cannot be represented exactly in binary
print(type(b))
print("{:30.20}".format(b)) #this is how it's actually storing it

<class 'float'>
        0.10000000000000000555


In [31]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

- rounding error cont:
    - consider the error to be a random number with standard deviation sigma=Cx, where C is called the error constant and is 10^-16 in python
    - then if we perform N summations the error would be (see slides)

In [37]:
x=1
y=1+(10**-14)*(2**0.5)
print(1e14*(y-x)) #mathematically this should give sqrt(2) but the subtraction is between numbers that are too close together
#to give an accurate difference between them
print(2**0.5)

1.4210854715202004
1.4142135623730951


- subtraction error:
    - if x=1000000000000000 and y=1000000000000001.25, y-x should be 1.25 but it will give 1
    - see slides for example of series involving addition and subtraction giving a huge error and how to handle it
    - ex: calculating the volume of a shell
        - uses the formula V=(4/3)*pi*(r1^3-r2^3) but this can cause problems because it involves the cancelation of two really big numbers
        - instead can rewrite the formula as V=(4/3)*pi*(r1-r2)*(r1^2+r1r2+r2^2) because then the subtraction is between smaller numbers

In [41]:
def quadraticformula(a,b,c):
    discriminant=b**2-4*a*c
    soln1=(-b+discriminant**0.5)/(2*a)
    soln2=(-b-discriminant**0.5)/(2*a)
    print(f'Solution 1: {soln1}, Solution 2: {soln2}')

In [43]:
quadraticformula(0.001,1000,0.001)

Solution 1: -9.999894245993346e-07, Solution 2: -999999.999999


In [49]:
def quadraticformula2(a,b,c):
    discriminant=b**2-4*a*c
    soln1=(2*c)/(-b-discriminant**0.5)
    soln2=(2*c)/(-b+discriminant**0.5)
    print(f'Solution 1: {soln1}, Solution 2: {soln2}')

In [51]:
quadraticformula2(0.001,1000,0.001)

Solution 1: -1.000000000001e-06, Solution 2: -1000010.5755125057


- speed: computers do not have infinite memory or speed
    - 1 million calculations takes about 1 second
    - 1 billion calculations takes about 20 minutes
    - 1 trillion calculations takes about 300 hours/12 days
    - often increasing the number of calculations does not increase the accuracy (eg, once you get past the smallest storable number with a taylor series) so don't want to do that and waste time
    - with matrix multiplication or manipulation, the number of operations increases very rapidly with N - in practice, can't really work with matrices larger than 1000x1000

exercise:
- solve the average energy in a quantum harmonic oscillator which has energy levels E_n=hf(n+1/2)
- average energy is given by 1/Z times the sum from 0 to infinity of E_n time e^(-beta* E_n) where Z is partition function and is the sum from 0 to infinity of e^(-beta* E_n)
- take hf=1 and beta=0.01 and evaluate this formula using a thousand terms, then 1 million and then 1 billion

In [59]:
import numpy as np

In [103]:
def E_n(n, hf=1): #n is the current term
    return hf*(n+1/2)

In [105]:
def Z(N, hf=1, beta=0.01): #N is number of terms so n=10 would go from 0 to 9
    mysum=0
    for n in range(0,N):
        mysum+=np.exp(-beta*E_n(n,hf))
    return mysum

In [107]:
def E(N, hf=1, beta=0.01):
    curZ=Z(N,hf,beta)
    mysum=0
    for n in range(0,N):
        curE_n=E_n(n,hf)
        mysum+=curE_n*np.exp(-beta*curE_n)
    return (1/curZ)*mysum

In [109]:
E(10)

4.917513884193951

In [111]:
E(1000) #much more accurate than 10 terms

99.95543134093475

In [117]:
E(1000000) #small but siginificant difference from 1000 terms and still very fast

100.00083333194436

In [119]:
E(1000000000)

KeyboardInterrupt: 

terminology for midterm:
- overflow error
- underflow error
- roundoff error

In [132]:
def f(x):
    return x**4-2*x+1

In [134]:
def trapezoid(function, a, b, N=10):
    deltax=(b-a)/N
    mysum=0
    for k in range(1,N+1):
        mysum+=function(a+k*deltax)
    fa=function(a)
    fb=function(b)
    return deltax*(0.5*fa+0.5*fb+mysum)

In [136]:
trapezoid(f,0,2)

7.106560000000001

In [138]:
trapezoid(f,0,2,N=100)

4.661066656000001

In [140]:
trapezoid(f,0,2,N=1000)

4.4260106666656

In [142]:
trapezoid(f,0,2,N=100000)

4.400260001066633